# DEA Opioid Data

- Downloaded from here on July 30, 2021 as gzipped .tsv: https://d2ty8gaf6rmowa.cloudfront.net/dea-pain-pill-database/bulk/arcos_all_washpost.tsv.gz (clicking starts download)
- Extracted and uploaded to **coiled-datasets** S3 bucket


In [1]:
import coiled
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
cluster = coiled.Cluster(
    name="data-import",
    software='rrpelgrim/coiled-base-m1',
    n_workers=20,
    worker_cpu=4,
    worker_memory='32Gb',
    scheduler_cpu=2,
    shutdown_on_close=False,
    scheduler_options={'idle_timeout':'1hour'},
)

Output()

Using existing cluster: 'data-import'

In [3]:
client = Client(cluster)
client

<Client: 'tls://10.4.0.67:8786' processes=20 threads=80, memory=580.00 GiB>

In [12]:
client.restart()

<Client: 'tls://10.4.0.67:8786' processes=20 threads=80, memory=580.00 GiB>

In [15]:
columns_selection = [
 #'REPORTER_DEA_NO',
 'REPORTER_BUS_ACT',
 #'REPORTER_NAME',
 #'REPORTER_ADDL_CO_INFO',
 #'REPORTER_ADDRESS1',
 #'REPORTER_ADDRESS2',
 #'REPORTER_CITY',
 #'REPORTER_STATE',
 #'REPORTER_ZIP',
 #'REPORTER_COUNTY',
 #'BUYER_DEA_NO',
 #'BUYER_BUS_ACT',
 #'BUYER_NAME',
 #'BUYER_ADDL_CO_INFO',
 #'BUYER_ADDRESS1',
 #'BUYER_ADDRESS2',
 #'BUYER_CITY',
 #'BUYER_STATE',
 #'BUYER_ZIP',
 #'BUYER_COUNTY',
 #'TRANSACTION_CODE',
 #'DRUG_CODE',
 #'NDC_NO',
 #'DRUG_NAME',
 #'QUANTITY',
 #'UNIT',
 #'ACTION_INDICATOR',
 #'ORDER_FORM_NO',
 #'CORRECTION_NO',
 #'STRENGTH',
 #'TRANSACTION_DATE',
 #'CALC_BASE_WT_IN_GM',
 #'DOSAGE_UNIT',
 #'TRANSACTION_ID',
 #'Product_Name',
 #'Ingredient_Name',
 #'Measure',
 #'MME_Conversion_Factor',
 #'Combined_Labeler_Name',
 #'Revised_Company_Name',
 #'Reporter_family',
 #'dos_str'
]

In [16]:
columns_selection

['REPORTER_BUS_ACT']

In [17]:
data = dd.read_table(
    "s3://coiled-datasets/dea-opioid/arcos_washpost.tsv",
    dtype={
 'REPORTER_DEA_NO': 'object',
 'REPORTER_BUS_ACT': 'object',
 'REPORTER_NAME': 'object',
 'REPORTER_ADDL_CO_INFO': 'object',
 'REPORTER_ADDRESS1': 'object',
 'REPORTER_ADDRESS2': 'object',
 'REPORTER_CITY': 'object',
 'REPORTER_STATE': 'object',
 'REPORTER_ZIP': 'int64',
 'REPORTER_COUNTY': 'object',
 'BUYER_DEA_NO': 'object',
 'BUYER_BUS_ACT': 'object',
 'BUYER_NAME': 'object',
 'BUYER_ADDL_CO_INFO': 'object',
 'BUYER_ADDRESS1': 'object',
 'BUYER_ADDRESS2': 'object',
 'BUYER_CITY': 'object',
 'BUYER_STATE': 'object',
 'BUYER_ZIP': 'int64',
 'BUYER_COUNTY': 'object',
 'TRANSACTION_CODE': 'object',
 'DRUG_CODE': 'int64',
 'NDC_NO': 'object',
 'DRUG_NAME': 'object',
 'QUANTITY': 'float64',
 'UNIT': 'object',
 'ACTION_INDICATOR': 'object',
 'ORDER_FORM_NO': 'object',
 'CORRECTION_NO': 'float64',
 'STRENGTH': 'float64',
 'TRANSACTION_DATE': 'int64',
 'CALC_BASE_WT_IN_GM': 'float64',
 'DOSAGE_UNIT': 'float64',
 'TRANSACTION_ID': 'int64',
 'Product_Name': 'object',
 'Ingredient_Name': 'object',
 'Measure': 'object',
 'MME_Conversion_Factor': 'float64',
 'Combined_Labeler_Name': 'object',
 'Revised_Company_Name': 'object',
 'Reporter_family': 'object',
 'dos_str': 'float64'
    },
    usecols=columns_selection,
)

In [18]:
data.head()

,REPORTER_BUS_ACT
0,DISTRIBUTOR
1,DISTRIBUTOR
2,DISTRIBUTOR
3,DISTRIBUTOR
4,DISTRIBUTOR


In [19]:
data = data.repartition(partition_size='100MB').persist()

In [20]:
data.npartitions

125

In [21]:
data.head()

,REPORTER_BUS_ACT
0,DISTRIBUTOR
1,DISTRIBUTOR
2,DISTRIBUTOR
3,DISTRIBUTOR
4,DISTRIBUTOR


In [8]:
%%time
data.groupby('REPORTER_CITY').DOSAGE_UNIT.mean().compute()

CPU times: user 545 ms, sys: 29.9 ms, total: 575 ms
Wall time: 20.6 s


REPORTER_CITY
BROCKTON              300.000000
COLUMBUS              237.618326
GOUVERNEUR            655.202560
GRANDVIEW HEIGHTS     797.290070
GUAYNABO              500.919179
                        ...     
EAST LIVERPOOL       5600.000000
SUGAR GROVE           100.000000
NEWTOWN              2800.000000
LINCOLN                 4.000000
NEW HYDE PARK           5.000000
Name: DOSAGE_UNIT, Length: 418, dtype: float64

In [ ]:
#data.dtypes.apply(lambda x: x.name).to_dict()

In [ ]:
#data.columns.to_list()

In [22]:
data.to_parquet(
    "s3://coiled-datasets/dea-opioid/tests/1-column.parquet"
)

KilledWorker: ("('to-parquet-3e34444024849a4da077afa2a10106e9', 85)", <WorkerState 'tls://10.4.1.44:45821', name: coiled-dask-rrpelgr71-34336-worker-48fa4f, memory: 0, processing: 4>)

In [12]:
import pandas as pd
print(pd.__version__)

1.3.1


In [ ]:
ddf_test = dd.read_parquet(
    "s3://coiled-datasets/dea-opioid/arcos_washpost.parquet"
)